In [25]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [26]:
model = load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder_geography.pkl', 'rb') as f:
    one_hot_encoder_geography = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [27]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [28]:
input_data = pd.DataFrame([input_data])
input_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,5,50000,2,1,1,60000


In [29]:
geo_encoded = one_hot_encoder_geography.transform([input_data['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))  
geo_encoded_df

c:\Users\Abhis\OneDrive\Desktop\Projects\Churn Modelling\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [30]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,5,50000,2,1,1,60000


In [31]:
input_data = pd.concat([input_data.drop(columns = ['Geography']), geo_encoded_df], axis=1)
input_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,5,50000,2,1,1,60000,1.0,0.0,0.0


In [32]:
input_data = input_data[['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain']]

In [33]:
input_scaled = scaler.transform(input_data)


In [34]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [35]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 73ms/step


array([[0.02339664]], dtype=float32)

In [36]:
pred_proba = prediction[0][0]

In [37]:
if pred_proba > 0.5:
    print(f"The customer is likely to churn with a probability of {pred_proba:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {pred_proba:.2f}")

The customer is unlikely to churn with a probability of 0.02
